In [27]:
%matplotlib notebook
import IPython
import librosa
import numpy as np
import soundfile
import subprocess
import matplotlib.pyplot as plt

from librosa import display

SR = 44100           # Sampling Rate (in Hz)
CHANNELS = 4         # Number of output channels
ENC = "s16le"        # Encoding (for FFMPEG)
OFFSET_SAMP = 7926      # Offset in samples TODO: Encontrar
INTER_SAMP = 130722  # Interleaved samples TODO: ENCONTRAR

# Paths
RAW_FILE = "/Users/onieto/Downloads/bm.raw"
WAV_FILE1 = "/Users/onieto/Downloads/bm1.wav"
WAV_FILE2 = "/Users/onieto/Downloads/bm2.wav"
TMP_FILE = "tmp.wav"

In [28]:
def raw_to_interleaved_wav():
    """
    Convert to WAV from Zoom's RAW format. 
    If more than 2 channels, result will be interleaved.
    """
    cmd = "ffmpeg -y -f {} -ss 0 -t 60 -ar {:.1f}k -ac 2 -i {} {}".format(
        ENC, SR / 1000, RAW_FILE, TMP_FILE)
    subprocess.call(cmd.split(" "))  

# Convert RAW to WAV using FFMPEG
raw_to_interleaved_wav()

In [ ]:
def make_even(x):
    return x if len(x) % 2 == 0 else x[:-1]

def to_mono(x):
    return x.flatten('F')

def to_stereo(x):
    return x.reshape((2, -1), order='F')
    
def interleaved_wav_to_wavs():
    # Read file
    x, fs = librosa.load(TMP_FILE, sr=SR, mono=False)
    
    # Read interleaved chunks
    x = to_mono(x[:, OFFSET_SAMP * 2:])
    chunk_size = int(INTER_SAMP * 2)
    y = np.array_split(x, np.arange(chunk_size, len(x), chunk_size))
    
    # De-interleave chunks and make even
    y1 = make_even(np.concatenate(y[1::2]))
    y2 = make_even(np.concatenate(y[::2]))
    
    # Return Stereo signals
    return to_stereo(y1), to_stereo(y2)
    
x1, x2 = interleaved_wav_to_wavs()
IPython.display.Audio(data=x1, rate=SR)
# soundfile.write(file=WAV_FILE,
#                 data=x1.T,
#                 samplerate=SR)

In [30]:
# 269370 -> 269371 # 2nd change
# 530814 -> 530815 # 4th change
# 1075694 -> 1075695 # 8th change
269370 / 2

134685.0

In [16]:
(530814 - 269370) / 2

130722.0

In [26]:
530814 - 130722 * 4

7926

In [31]:
(1075694 - 530814) / 4

136220.0

In [34]:
(1075694 - 269370) / 6

134387.33333333334